In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
import tensorflow as tf

In [3]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, GRU
from tensorflow.keras import Input
from tensorflow.keras.losses import sparse_categorical_crossentropy

## Import Data

In [4]:
path = 'C:\\Users\\alvar\\Documents\\tf_templates\\DATA\\Quijote.txt'

In [5]:
with open(path, 'r', encoding='utf-8') as file:
    text = file.read()

In [6]:
print(text[:500])

El ingenioso hidalgo don Quijote de la Mancha
por Miguel de Cervantes Saavedra
El ingenioso hidalgo don Quijote de la Mancha
Tasa
Testimonio de las erratas
El Rey
Al Duque de Béjar
Prólogo
Al libro de don Quijote de la Mancha
Que trata de la condición y ejercicio del famoso hidalgo don Quijote de la Mancha
Que trata de la primera salida que de su tierra hizo el ingenioso don Quijote
Donde se cuenta la graciosa manera que tuvo don Quijote en armarse caballero
De lo que le sucedió a nuestro caball


In [7]:
# The unique characters in the file
vocab = sorted(set(text))
print(vocab)
len(vocab)

['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í', 'Ñ', 'Ó', 'Ú', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü', '—']


92

## Text Processing

In [8]:
char_to_ind = {u:i for i, u in enumerate(vocab)}
char_to_ind

{'\n': 0,
 ' ': 1,
 '!': 2,
 '"': 3,
 "'": 4,
 '(': 5,
 ')': 6,
 ',': 7,
 '-': 8,
 '.': 9,
 '0': 10,
 '1': 11,
 '2': 12,
 '3': 13,
 '4': 14,
 '5': 15,
 '6': 16,
 '7': 17,
 ':': 18,
 ';': 19,
 '?': 20,
 'A': 21,
 'B': 22,
 'C': 23,
 'D': 24,
 'E': 25,
 'F': 26,
 'G': 27,
 'H': 28,
 'I': 29,
 'J': 30,
 'L': 31,
 'M': 32,
 'N': 33,
 'O': 34,
 'P': 35,
 'Q': 36,
 'R': 37,
 'S': 38,
 'T': 39,
 'U': 40,
 'V': 41,
 'W': 42,
 'X': 43,
 'Y': 44,
 'Z': 45,
 ']': 46,
 'a': 47,
 'b': 48,
 'c': 49,
 'd': 50,
 'e': 51,
 'f': 52,
 'g': 53,
 'h': 54,
 'i': 55,
 'j': 56,
 'l': 57,
 'm': 58,
 'n': 59,
 'o': 60,
 'p': 61,
 'q': 62,
 'r': 63,
 's': 64,
 't': 65,
 'u': 66,
 'v': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '¡': 71,
 '«': 72,
 '»': 73,
 '¿': 74,
 'Á': 75,
 'É': 76,
 'Í': 77,
 'Ñ': 78,
 'Ó': 79,
 'Ú': 80,
 'à': 81,
 'á': 82,
 'é': 83,
 'í': 84,
 'ï': 85,
 'ñ': 86,
 'ó': 87,
 'ù': 88,
 'ú': 89,
 'ü': 90,
 '—': 91}

In [9]:
ind_to_char = np.array(vocab)
ind_to_char

array(['\n', ' ', '!', '"', "'", '(', ')', ',', '-', '.', '0', '1', '2',
       '3', '4', '5', '6', '7', ':', ';', '?', 'A', 'B', 'C', 'D', 'E',
       'F', 'G', 'H', 'I', 'J', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S',
       'T', 'U', 'V', 'W', 'X', 'Y', 'Z', ']', 'a', 'b', 'c', 'd', 'e',
       'f', 'g', 'h', 'i', 'j', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's',
       't', 'u', 'v', 'x', 'y', 'z', '¡', '«', '»', '¿', 'Á', 'É', 'Í',
       'Ñ', 'Ó', 'Ú', 'à', 'á', 'é', 'í', 'ï', 'ñ', 'ó', 'ù', 'ú', 'ü',
       '—'], dtype='<U1')

In [10]:
encoded_text = np.array([char_to_ind[c] for c in text])
encoded_text

array([25, 57,  1, ..., 26, 55, 59])

In [11]:
sample = text[:20]
sample

'El ingenioso hidalgo'

In [12]:
encoded_text[:20]

array([25, 57,  1, 55, 59, 53, 51, 59, 55, 60, 64, 60,  1, 54, 55, 50, 47,
       57, 53, 60])

## Creating Batches

In [13]:
print(text[:500])

El ingenioso hidalgo don Quijote de la Mancha
por Miguel de Cervantes Saavedra
El ingenioso hidalgo don Quijote de la Mancha
Tasa
Testimonio de las erratas
El Rey
Al Duque de Béjar
Prólogo
Al libro de don Quijote de la Mancha
Que trata de la condición y ejercicio del famoso hidalgo don Quijote de la Mancha
Que trata de la primera salida que de su tierra hizo el ingenioso don Quijote
Donde se cuenta la graciosa manera que tuvo don Quijote en armarse caballero
De lo que le sucedió a nuestro caball


In [14]:
line = '— Yo me acuerdo haber leído que un caballero español, llamado Diego Pérez de Vargas, habiéndosele en una batalla roto la espada, desgajó '

In [15]:
len(line)

137

In [16]:
paragraph = '— Yo me acuerdo haber leído que un caballero español, llamado Diego Pérez de Vargas, habiéndosele en una batalla roto la espada, desgajó de una encina un pesado ramo o tronco, y con él hizo tales cosas aquel día, y machacó tantos moros, que le quedó por sobrenombre Machuca, y así él como sus decendientes se llamaron, desde aquel día en adelante, Vargas y Machuca. Hete dicho esto, porque de la primera encina o roble que se me depare pienso desgajar otro tronco tal y tan bueno como aquél, que me imagino y pienso hacer con él tales hazañas, que tú te tengas por bien afortunado de haber merecido venir a vellas y a ser testigo de cosas que apenas podrán ser creídas.'

In [17]:
len(paragraph)

669

## Training Sequence

In [18]:
seq_len = 140
total_num_seq = len(text) // (seq_len + 1)
total_num_seq

14964

In [19]:
# Create Training Sequences
char_dataset = tf.data.Dataset.from_tensor_slices(encoded_text)

for i in char_dataset.take(500):
     print(ind_to_char[i.numpy()])

E
l
 
i
n
g
e
n
i
o
s
o
 
h
i
d
a
l
g
o
 
d
o
n
 
Q
u
i
j
o
t
e
 
d
e
 
l
a
 
M
a
n
c
h
a


p
o
r
 
M
i
g
u
e
l
 
d
e
 
C
e
r
v
a
n
t
e
s
 
S
a
a
v
e
d
r
a


E
l
 
i
n
g
e
n
i
o
s
o
 
h
i
d
a
l
g
o
 
d
o
n
 
Q
u
i
j
o
t
e
 
d
e
 
l
a
 
M
a
n
c
h
a


T
a
s
a


T
e
s
t
i
m
o
n
i
o
 
d
e
 
l
a
s
 
e
r
r
a
t
a
s


E
l
 
R
e
y


A
l
 
D
u
q
u
e
 
d
e
 
B
é
j
a
r


P
r
ó
l
o
g
o


A
l
 
l
i
b
r
o
 
d
e
 
d
o
n
 
Q
u
i
j
o
t
e
 
d
e
 
l
a
 
M
a
n
c
h
a


Q
u
e
 
t
r
a
t
a
 
d
e
 
l
a
 
c
o
n
d
i
c
i
ó
n
 
y
 
e
j
e
r
c
i
c
i
o
 
d
e
l
 
f
a
m
o
s
o
 
h
i
d
a
l
g
o
 
d
o
n
 
Q
u
i
j
o
t
e
 
d
e
 
l
a
 
M
a
n
c
h
a


Q
u
e
 
t
r
a
t
a
 
d
e
 
l
a
 
p
r
i
m
e
r
a
 
s
a
l
i
d
a
 
q
u
e
 
d
e
 
s
u
 
t
i
e
r
r
a
 
h
i
z
o
 
e
l
 
i
n
g
e
n
i
o
s
o
 
d
o
n
 
Q
u
i
j
o
t
e


D
o
n
d
e
 
s
e
 
c
u
e
n
t
a
 
l
a
 
g
r
a
c
i
o
s
a
 
m
a
n
e
r
a
 
q
u
e
 
t
u
v
o
 
d
o
n
 
Q
u
i
j
o
t
e
 
e
n
 
a
r
m
a
r
s
e
 
c
a
b
a
l
l
e
r
o


D
e
 
l
o
 
q
u
e
 
l
e
 
s
u
c
e
d
i
ó
 
a
 
n
u
e
s
t
r
o
 
c
a
b
a
l
l


In [20]:
sequences = char_dataset.batch(seq_len+1, drop_remainder=True)

In [21]:
def create_seq_targets(seq):
    input_txt = seq[:-1]
    target_txt = seq[1:]
    return input_txt, target_txt

In [22]:
dataset = sequences.map(create_seq_targets)

In [23]:
for input_txt, target_txt in  dataset.take(1):
    print(input_txt.numpy())
    print(''.join(ind_to_char[input_txt.numpy()]))
    print('\n')
    print(target_txt.numpy())
    # There is an extra whitespace!
    print(''.join(ind_to_char[target_txt.numpy()]))

[25 57  1 55 59 53 51 59 55 60 64 60  1 54 55 50 47 57 53 60  1 50 60 59
  1 36 66 55 56 60 65 51  1 50 51  1 57 47  1 32 47 59 49 54 47  0 61 60
 63  1 32 55 53 66 51 57  1 50 51  1 23 51 63 67 47 59 65 51 64  1 38 47
 47 67 51 50 63 47  0 25 57  1 55 59 53 51 59 55 60 64 60  1 54 55 50 47
 57 53 60  1 50 60 59  1 36 66 55 56 60 65 51  1 50 51  1 57 47  1 32 47
 59 49 54 47  0 39 47 64 47  0 39 51 64 65 55 58 60 59 55 60]
El ingenioso hidalgo don Quijote de la Mancha
por Miguel de Cervantes Saavedra
El ingenioso hidalgo don Quijote de la Mancha
Tasa
Testimonio


[57  1 55 59 53 51 59 55 60 64 60  1 54 55 50 47 57 53 60  1 50 60 59  1
 36 66 55 56 60 65 51  1 50 51  1 57 47  1 32 47 59 49 54 47  0 61 60 63
  1 32 55 53 66 51 57  1 50 51  1 23 51 63 67 47 59 65 51 64  1 38 47 47
 67 51 50 63 47  0 25 57  1 55 59 53 51 59 55 60 64 60  1 54 55 50 47 57
 53 60  1 50 60 59  1 36 66 55 56 60 65 51  1 50 51  1 57 47  1 32 47 59
 49 54 47  0 39 47 64 47  0 39 51 64 65 55 58 60 59 55 60  1]
l i

## Generating Training Batches

In [24]:
# Batch size
batch_size = 128

# Buffer size to shuffle the dataset so it doesn't attempt to shuffle
# the entire sequence in memory. Instead, it maintains a buffer in which it shuffles elements
buffer_size = 10000

dataset = dataset.shuffle(buffer_size).batch(batch_size, drop_remainder=True)

## Creating the Model

In [25]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embed_dim = 10

# Number of RNN units
rnn_neurons = 4

In [26]:
def sparse_cat_loss(y_true,y_pred):
  return sparse_categorical_crossentropy(y_true, y_pred, from_logits=True)

In [32]:
def create_model(vocab_size, embed_dim, rnn_neurons, weights=None):
    model = Sequential()
    model.add(Embedding(input_dim=vocab_size, output_dim=embed_dim, weights=None))
    model.add(GRU(rnn_neurons,return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'))
    # Final Dense Layer to Predict
    model.add(Dense(vocab_size))
    model.compile(optimizer='adam', loss=sparse_cat_loss) 
    return model

In [33]:
model = create_model(
  vocab_size = vocab_size,
  embed_dim=embed_dim,
  rnn_neurons=rnn_neurons)

ValueError: Unrecognized keyword arguments passed to Embedding: {'weights': None}

## Training the Model

In [ ]:
for input_example_batch, target_example_batch in dataset.take(1):

  # Predict off some random batch
  example_batch_predictions = model(input_example_batch)

  # Display the dimensions of the predictions
  print(example_batch_predictions.shape, " <=== (batch_size, sequence_length, vocab_size)")

In [ ]:
example_batch_predictions

In [ ]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)

In [ ]:
sampled_indices

In [ ]:
# Reformat to not be a lists of lists
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [ ]:
sampled_indices

In [ ]:
print("Given the input seq: \n")
print("".join(ind_to_char[input_example_batch[0]]))
print('\n')
print("Next Char Predictions: \n")
print("".join(ind_to_char[sampled_indices ]))

In [ ]:
epochs = 1

In [ ]:
model.fit(dataset, epochs=epochs)

In [ ]:
model.save_weights('C:\\Users\\alvar\\Documents\\tf_templates\\NLP\\quijote_model.weights.h5')

## Generating Text

In [ ]:
'''Currently our model only expects 128 sequences at a time. We can create a new model
that only expects a batch_size=1. We can create a new model with this batch size, then
load our saved models weights. Then call .build() on the model:'''

model = create_model(vocab_size, embed_dim, rnn_neurons, batch_size=1)

model.load_weights('C:\\Users\\alvar\\Documents\\tf_templates\\NLP\\quijote_model.weights.h5')

model.build(tf.TensorShape([1, None]))